In [1]:
from scipy import optimize
from bingo.evaluation.fitness_function import VectorBasedFunction
import numpy as np
import sys
from numpy import array

In [9]:
number_of_eqps_values = 10
P_desired = np.array([[1, -0.5, -0.5],
                    [-0.5, 1, -0.5],
                    [-0.5, -0.5, 1]])
dataloc = 'processed_data/hill_w_hardening.txt'
pre_data = np.loadtxt(dataloc)
#print(pre_data)
training_data = []
eqpses = []
for row in pre_data:
    if np.isnan(row[0]) == False:
        training_data.append(row[:3])
        eqpses.append(row[3])
training_data = np.array(training_data)
eqpses = np.array(eqpses)
#print(eqpses)
#print(training_data)

def mapping_(x, return_error=True):

    # mapping = lambda X_0: array([[x[0], x[1], x[2]],
    #    [x[3], x[4], x[5]],
    #    [x[6], x[7], x[8]]]) + (X_0) * (array([[x[9], x[10], x[11]],
    #    [x[12], x[12], x[13]],
    #    [x[14], x[15], x[16]]]))
    mapping = lambda X_0: ((array(x[0]))/(X_0 + array(x[1])) ) * (array([[x[2], x[3],  x[4]],
       [x[5], x[6],  x[7] ],
       [x[8], x[9], x[10] ]]))
    # mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], 0, 0 ],  
    #     [0 , x[6], 0],
    #     [0, 0, x[7]]]))
    # ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],
    #    [-0.8733965 , -1.02045703, -0.37967495],
    #    [-0.86253049, -1.02066836, -0.38872487]]))
    #mapping = lambda X_0: ((x[0]*X_0 + x[1]) )*((x[2]*X_0 + x[3]) ) * np.eye(3)
    
    # (array([[x[2], x[3], x[4] ],  
    #     [x[3] , x[6], x[5]],
    #     [x[4], x[5], x[7]]]))
    
    mapping_matrices = np.array([mapping(e) for e in eqpses])
    P_mapped = np.transpose(mapping_matrices,(0, 2, 1)) @ P_desired @ mapping_matrices
    back_calculated_yield_stresses = []
    for i,stresses in enumerate(training_data):
        #print('here we go', np.transpose(training_data[i]), P_mapped[i], training_data[i])
        back_calculated_yield_stresses.append(np.transpose(training_data[i]) @ P_mapped[i] @ training_data[i])
        #back_calculated_yield_stresses.append(P_mapped[i] @ training_data[i])
        #print(P_mapped[i] @ training_data[i])
    #print(back_calculated_yield_stresses)
    back_calculated_yield_stresses = np.array(back_calculated_yield_stresses)
    # Y_0 = 10
    # H = 990
    y = np.array([ 1 for eqps_ in eqpses ])
    #print(back_calculated_yield_stresses.shape, y.shape)
    error = y - back_calculated_yield_stresses
    #error = training_data - np.array(back_calculated_yield_stresses)
    #print(error)
    if return_error:
        return error.flatten()
    else:
        return back_calculated_yield_stresses, y, mapping_matrices


# ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],  
#        [-0.8733965 , -1.02045703, -0.37967495],
#        [-0.86253049, -1.02066836, -0.38872487]]))


In [10]:
x0 = [0.19468542, 0.19468542, -0.78883927, -0.17774928, 0.05046644, -0.94038756, -0.01281956, 0.0229076, -0.80953323, -0.10425625, 
     -0.02368224 ]
#x0 = [ 1, 1, 1, -0.3899734, -0.8733965 , -1.02045703, -0.37967495 ]

mapping_(x0)
#results = optimize.minimize(mapping_, )

array([ 5.61631025e-03,  5.75139839e-03,  5.79121369e-03,  5.70433607e-03,
        5.66491197e-03,  5.73973949e-03,  5.69765068e-03,  5.65770984e-03,
        5.61739528e-03,  5.57814278e-03,  5.49648501e-03,  5.64360192e-03,
        5.73619965e-03,  5.69791617e-03,  5.67121376e-03,  5.41526941e-03,
        5.37388527e-03,  5.33516166e-03,  5.32394808e-03,  5.31357642e-03,
        4.37029324e-03,  4.52413946e-03,  4.62942128e-03,  4.68795886e-03,
        4.64906337e-03,  4.09109853e-03,  4.08034842e-03,  4.02418374e-03,
        4.06128962e-03,  4.03614377e-03,  2.33759838e-03,  2.45866711e-03,
        2.72136316e-03,  2.79068439e-03,  2.72116603e-03,  1.99905160e-03,
        2.06927088e-03,  1.99231814e-03,  2.04239431e-03,  2.01078261e-03,
        6.65240955e-04,  7.58347140e-04,  9.39097062e-04,  1.01655726e-03,
        9.67502463e-04,  4.28040854e-04,  4.46766447e-04,  3.48104715e-04,
        4.39723838e-04,  3.79838103e-04, -6.30179897e-04, -5.79960117e-04,
       -3.69905732e-04, -

In [11]:
results = optimize.root(mapping_, x0=x0, method='lm', tol=1e-16, options={"ftol": 1e-7, "gtol": 1e-16, "maxiter": 100000})
results

   cov_x: array([[ 1.95529751e+10, -8.56715705e+02, -6.79243485e+10,
        -5.17637799e+09, -1.49338114e+10, -1.71563580e+10,
         2.14565603e+09,  1.13014473e+09, -3.02397171e+10,
        -2.93214916e+10, -1.40601394e+09],
       [-8.71939294e+02,  5.02995269e-04,  1.83952110e+03,
        -3.98298232e+02,  4.07418725e+02,  4.62798862e+02,
        -6.11838178e+01,  1.45318526e+02,  8.50837757e+02,
         3.50313786e+02, -8.12410605e+01],
       [-7.99498004e+10,  2.47306392e+03,  4.29206638e+11,
         1.45472021e+11,  1.35531890e+11,  1.32030674e+09,
        -4.37574879e+10, -3.23056920e+10,  6.05206121e+10,
         3.07921739e+11,  2.31346389e+10],
       [-2.12357278e+10,  4.92945171e+02,  1.93921701e+11,
         1.37520522e+11,  9.51418083e+10, -8.93898242e+10,
        -3.74510434e+10, -3.20529852e+10, -6.67465553e+10,
         2.22979183e+11,  1.85942681e+10],
       [-2.41090716e+10,  9.28726046e+02,  1.55184777e+11,
         8.50920669e+10,  7.21584518e+10, -3.770119

In [12]:
np.set_printoptions(threshold=sys.maxsize)
error_array = mapping_(results.x)
print(error_array, '\nMax:', np.max(error_array))

[-1.58891884e-04  1.14908646e-05  9.62681876e-05  8.93385938e-05
  5.98689053e-05  2.95543477e-05 -1.79552908e-06 -3.67987159e-05
 -6.80353107e-05 -9.88296398e-05 -1.27428489e-04  2.11895339e-05
  1.08336748e-04  9.34320591e-05  6.58401644e-05  3.30026135e-05
 -3.33572871e-06 -3.49801072e-05 -6.92865417e-05 -9.82917864e-05
 -1.29446556e-04  3.10756303e-05  1.13709890e-04  9.65766877e-05
  6.81160904e-05  3.40034559e-05 -3.24445051e-07 -3.14758437e-05
 -6.61013042e-05 -1.00588104e-04 -1.31174418e-04  3.64618637e-05
  1.10513041e-04  9.41269008e-05  6.81504962e-05  3.02894595e-05
  2.01979882e-06 -3.18256115e-05 -6.67897841e-05 -9.65817333e-05
 -1.28117462e-04  3.79143293e-05  1.08353156e-04  9.25435623e-05
  6.45552720e-05  3.04390068e-05 -3.24906599e-06 -3.63272601e-05
 -6.67139652e-05 -9.76629068e-05 -1.28247860e-04  1.07944624e-05
  1.02317960e-04  8.95472838e-05  6.85472795e-05  3.37907613e-05
  2.99002606e-06 -3.37971443e-05 -6.44039486e-05 -1.00197215e-04
 -1.32178007e-04  3.98232

In [13]:
np.set_printoptions(precision=32)
results.x

array([ 0.19395790241796493 ,  0.19466451043426053 ,
       -0.785455772704798   , -0.18131732698525171 ,
        0.04831612560296698 , -0.9443873169242883  ,
       -0.02171985979340752 ,  0.015920102302059713,
       -0.8102174075929641  , -0.10561489700080459 ,
       -0.027066650100274783])

In [28]:
predicted_YS, actual_Y, mappings = mapping_(results.x, return_error=False)

In [29]:
predicted_YS

array([  100.00002516825222,   396.0100680537548 ,   888.0400990542992 ,
        1576.0901063882775 ,  2460.160090390662  ,  3540.250058628921  ,
        4816.360021842414  ,  6288.48999070527   ,  7956.63997341273   ,
        9820.809974091015  , 11880.999992030616  , 14137.210021743096  ,
       16589.44005384137   , 19237.690076743493  , 22081.960079199842  ,
       25122.250053643897  , 28358.56000036641   , 31790.88993251307   ,
       35419.23988190572   , 39243.609905686964  , 43264.00009378824   ,
         100.00002516821374,   396.0100680536117 ,   888.0400990539969 ,
        1576.090106387771  ,  2460.1600903899143 ,  3540.2500586279    ,
        4816.360021841092  ,  6288.48999070362   ,  7956.639973410725  ,
        9820.80997408862   , 11880.999992027793  , 14137.21002173981   ,
       16589.440053837563  , 19237.69007673909   , 22081.96007919479   ,
       25122.25005363808   , 28358.56000035974   , 31790.889932505383  ,
       35419.23988189689   , 39243.6099056768    , 

In [30]:
actual_Y

array([  100.              ,   396.00999999999993,   888.0400000000001 ,
        1576.0900000000001 ,  2460.1600000000003 ,  3540.25            ,
        4816.360000000001  ,  6288.490000000002  ,  7956.64            ,
        9820.81            , 11881.              , 14137.210000000001  ,
       16589.440000000002  , 19237.690000000006  , 22081.960000000006  ,
       25122.25            , 28358.56            , 31790.890000000003  ,
       35419.24            , 39243.61            , 43264.              ,
         100.              ,   396.00999999999993,   888.0400000000001 ,
        1576.0900000000001 ,  2460.1600000000003 ,  3540.25            ,
        4816.360000000001  ,  6288.490000000002  ,  7956.64            ,
        9820.81            , 11881.              , 14137.210000000001  ,
       16589.440000000002  , 19237.690000000006  , 22081.960000000006  ,
       25122.25            , 28358.56            , 31790.890000000003  ,
       35419.24            , 39243.61            , 

In [31]:
mappings

array([[[-0.4235775835834732  , -0.04148242995369289 ,
          0.5943025397254625  ],
        [ 0.5770764819925227  ,  0.05778373947936392 ,
         -0.4019021627391064  ],
        [ 0.6422532886726127  , -0.9725103257468183  ,
          0.564662540091747   ]],

       [[-0.421936971517533   , -0.03849145955607853 ,
          0.5952166729421003  ],
        [ 0.5790056663964729  ,  0.06090900043062799 ,
         -0.4007090556302415  ],
        [ 0.6436824587076939  , -0.9697403958249298  ,
          0.5655099146149166  ]],

       [[-0.4204260358033526  , -0.03573689978681216 ,
          0.5960585517464875  ],
        [ 0.5807823652734848  ,  0.06378723623385679 ,
         -0.39961025343376166 ],
        [ 0.6449986650751042  , -0.9671894051723285  ,
          0.5662903114295712  ]],

       [[-0.419044776440932   , -0.03321875064589379 ,
          0.5968281761386237  ],
        [ 0.5824065786235585  ,  0.06641844688905034 ,
         -0.3986057561496668  ],
        [ 0.64620190777484